## Before Lecture

- Best assignment 1 presentation
- Assignment 1 marking does not depend on Kaggle. The prize does depend, but not only on Kaggle.
- quiz handwriting
- Project ideas/proposals
  - Helix Personas are inferred. They are endogenous. Usef with caution.
  - Survey methods (related to Covid) related bias.
  - Are people happier? how are they related to economic factors and vice versa.
  - [RM] Forecasting. E.g, car opinions -> car purchase in the future.
  - [RM] link to productivity/GDP. Attitude/sentiment from micro data -> macroeconomy
  - [RM] health and economics
  - inflation, unemployment, etc
- Opportunity to visit Roy Morgan in Week 5 (uncertain, depending on progress and both sides' schedules)

# House Keeping

In [2]:
import pandas as pd
import numpy as np
import plotly.express as px # pyright: ignore[reportMissingImports] t
import plotly.graph_objects as go # pyright: ignore[reportMissingImports] t
from sklearn.pipeline import Pipeline # pyright: ignore[reportMissingModuleSource]
from sklearn.linear_model import Lasso, LinearRegression, LogisticRegression, LassoCV, ElasticNetCV, ElasticNet # pyright: ignore[reportMissingModuleSource]
from sklearn.model_selection import train_test_split, GridSearchCV # pyright: ignore[reportMissingModuleSource]
from sklearn.preprocessing import StandardScaler # pyright: ignore[reportMissingModuleSource]
from sklearn.metrics import accuracy_score,mean_squared_error, make_scorer # pyright: ignore[reportMissingModuleSource]
import statsmodels.api as sm # pyright: ignore[reportMissingImports] 

import sy_functions as sy

You may find the function file [here](https://www.dropbox.com/s/zzz9ddc5dfknew9/sy_functions.py?dl=1) and upload it to your google drive.

# Reading

Textbook Page 77-104.

# LASSO

- Which variables are signals, and which are noise?


Now we proceed to modern techniques. Do you know that a simple MLE estimation may look like
$$\widehat{\beta}=\arg\min \left\{-\frac{2}{n}\log \text{lhd}(\beta)\right\},$$
where $\text{lhd}(\beta)$ is the likelihood function?

$\beta$ is so free that it can be wild. Such freedom needs to be regularised, especially when the parameter space is large! The principle is "I would rather have nothing than too much noise".

**Penealised deviance** is
$$\widehat{\beta}=\arg\min \left\{-\frac{2}{n}\log \text{lhd}(\beta)\right\} + \lambda\sum\limits_k c(\beta_k).$$

It is LASSO when the cost function $c(\beta_k)=|\beta_k|$. Or simply
$$\widehat{\beta}=\arg\min \left\{-\frac{2}{n}\log \text{lhd}(\beta)\right\} + \lambda\sum\limits_k |\beta_k|.$$



A simple illustration:
- deviance of a linear model is quadratic in $\beta$. Say $(\beta-b)^2$ for simplicity (over-simplified), where $b\neq 0$ is the MLE estimate.
- The penalty term is $|\beta|$ by letting $\lambda=1$ for simplicity.
- The objective function is
$$f = (\beta-b)^2+|\beta|$$
- If the signal $|b|$ (MLE estimate) is small, then the penalty term dominates. Because if choosing $\widehat{\beta}=b$, the loss is $|b|$; while the loss is a much smaller $b^2$ if setting $\beta=0$. In this case, if $|b|< 1$, we should let $\widehat{\beta}=0$.
- Otherwise, the likelihood part dominates.
Say if $|b|>1$, we should let $\widehat{\beta}$ to be closer to $b$ instead of $0$.

Figure 3.5
<img src="https://www.dropbox.com/s/v019wzxivoe4nkb/LASSO.png?dl=1" >


## Semiconductors

In [3]:
df_sc = pd.read_csv("https://www.dropbox.com/s/hl26fuodi5zalo5/semiconductor.csv?dl=1")
df_sc

,FAIL,SIG1,SIG2,SIG3,SIG4,SIG5,SIG6,SIG7,SIG8,SIG9,...,SIG191,SIG192,SIG193,SIG194,SIG195,SIG196,SIG197,SIG198,SIG199,SIG200
0,0,2.223779,-0.349018,-1.985797,-2.600993,0.451092,0.856625,0.019999,5.552095,-1.217598,...,-0.493650,-0.081946,-0.026963,0.069390,0.000884,-0.090158,0.567668,0.136708,0.258290,-0.110619
1,1,-0.130378,-0.381318,-1.285172,-0.766302,-3.098302,1.257852,0.792258,3.052683,-0.596629,...,-0.242341,-0.040653,-0.248808,-0.206819,-0.113580,0.087156,-0.075984,0.359806,0.352307,-0.065168
2,0,-0.871138,-0.825300,-1.615008,-0.448803,-2.570782,-1.345798,3.561107,2.748009,-0.898133,...,-0.210873,-0.224043,-0.668168,-0.067575,-0.557790,0.050106,0.352167,-0.136979,0.270314,0.231886
3,0,-0.877341,-1.905598,-2.306114,-1.210761,-4.037471,-2.223608,2.404128,0.769476,5.605738,...,-0.244826,0.100125,-0.016613,0.139226,0.153687,-0.222902,-0.324682,0.330688,-0.053124,-0.046525
4,0,-2.113456,-2.313845,-2.917153,-3.333467,-4.439346,3.029632,-0.177130,0.268809,2.673137,...,0.523039,-0.104421,0.110383,0.099062,0.347613,-0.213793,0.090675,0.081936,-0.129969,-0.435595
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1472,0,-1.216544,2.933523,3.125961,6.299250,-4.701302,-0.736551,3.274427,-2.604696,-1.287880,...,0.137749,-0.057304,-0.005772,0.044957,0.014158,-0.256404,0.094116,0.003980,-0.392673,-0.156949
1473,0,1.519838,0.929747,-1.755750,-0.627124,0.134864,-0.277557,0.433767,2.599539,0.523947,...,-0.070191,-0.446749,-0.038911,0.276027,0.146811,0.092740,-0.012873,0.173764,0.011626,-0.140552
1474,0,1.186808,3.124830,1.554038,7.640257,-2.306926,-0.103164,-0.717412,-0.002707,-1.555463,...,0.104973,0.738707,0.183454,-0.538684,0.066061,-0.045803,-0.223485,-0.244998,0.270780,-0.206376
1475,0,0.418682,-0.560838,-2.849019,-2.070624,-2.299557,4.606691,-2.472302,1.304957,-0.820954,...,0.235377,0.323977,-0.046804,0.033842,-0.230825,0.045902,0.093716,0.194718,-0.463204,0.210923


In [4]:
X0 = df_sc.drop(columns='FAIL')
y0 = df_sc['FAIL']

In [4]:
lasso_model = Lasso(alpha=0.01)  # You can adjust the alpha parameter for stronger or weaker regularization

lasso_model.fit(X0, y0)

,alpha,0.01
,fit_intercept,True
,precompute,False
,copy_X,True
,max_iter,1000
,tol,0.0001
,warm_start,False
,positive,False
,random_state,None
,selection,'cyclic'


In [5]:
coefficients = lasso_model.coef_
feature_names = X0.columns

print("Lasso Regression Coefficients:")
for feature, coef in zip(feature_names, coefficients):
    print(f"{feature}: {coef:.4f}")

Lasso Regression Coefficients:
SIG1: -0.0016
SIG2: -0.0073
SIG3: 0.0000
SIG4: -0.0014
SIG5: 0.0013
SIG6: 0.0000
SIG7: -0.0037
SIG8: 0.0000
SIG9: -0.0000
SIG10: 0.0009
SIG11: 0.0022
SIG12: 0.0003
SIG13: -0.0087
SIG14: -0.0023
SIG15: 0.0000
SIG16: 0.0031
SIG17: -0.0003
SIG18: 0.0056
SIG19: 0.0000
SIG20: -0.0026
SIG21: -0.0001
SIG22: -0.0004
SIG23: 0.0019
SIG24: 0.0060
SIG25: 0.0005
SIG26: -0.0000
SIG27: -0.0032
SIG28: 0.0000
SIG29: -0.0015
SIG30: -0.0000
SIG31: -0.0000
SIG32: -0.0027
SIG33: -0.0000
SIG34: -0.0005
SIG35: -0.0001
SIG36: -0.0000
SIG37: 0.0000
SIG38: 0.0000
SIG39: -0.0017
SIG40: 0.0006
SIG41: -0.0023
SIG42: 0.0042
SIG43: -0.0000
SIG44: -0.0000
SIG45: 0.0020
SIG46: 0.0000
SIG47: -0.0000
SIG48: -0.0017
SIG49: -0.0000
SIG50: 0.0089
SIG51: 0.0000
SIG52: -0.0000
SIG53: -0.0000
SIG54: -0.0000
SIG55: -0.0000
SIG56: -0.0000
SIG57: -0.0056
SIG58: -0.0000
SIG59: 0.0026
SIG60: -0.0015
SIG61: -0.0112
SIG62: -0.0006
SIG63: 0.0000
SIG64: 0.0024
SIG65: -0.0000
SIG66: 0.0027
SIG67: -0.0000


In [6]:
sum(coefficients!=0)

52

## Scaling

In [5]:
scaler = StandardScaler()
X1 = scaler.fit_transform(X0)

In [8]:
lasso_model1 = Lasso(alpha=0.01)  # You can adjust the alpha parameter for stronger or weaker regularization

lasso_model1.fit(X1, y0)

,alpha,0.01
,fit_intercept,True
,precompute,False
,copy_X,True
,max_iter,1000
,tol,0.0001
,warm_start,False
,positive,False
,random_state,None
,selection,'cyclic'


In [9]:
sum(lasso_model1.coef_ != 0)

47

In [10]:
coefficients = lasso_model1.coef_
feature_names = X0.columns

print("Lasso Regression Coefficients:")
for feature, coef in zip(feature_names, coefficients):
    print(f"{feature}: {coef:.4f}")

Lasso Regression Coefficients:
SIG1: -0.0000
SIG2: -0.0222
SIG3: 0.0000
SIG4: -0.0000
SIG5: 0.0000
SIG6: 0.0000
SIG7: -0.0041
SIG8: 0.0000
SIG9: -0.0000
SIG10: 0.0000
SIG11: 0.0000
SIG12: 0.0000
SIG13: -0.0147
SIG14: -0.0000
SIG15: 0.0000
SIG16: 0.0014
SIG17: -0.0000
SIG18: 0.0068
SIG19: 0.0000
SIG20: -0.0001
SIG21: -0.0000
SIG22: -0.0000
SIG23: 0.0000
SIG24: 0.0072
SIG25: 0.0000
SIG26: -0.0000
SIG27: -0.0014
SIG28: 0.0000
SIG29: -0.0000
SIG30: -0.0000
SIG31: -0.0000
SIG32: -0.0004
SIG33: -0.0000
SIG34: -0.0000
SIG35: -0.0000
SIG36: -0.0000
SIG37: 0.0000
SIG38: 0.0000
SIG39: -0.0000
SIG40: 0.0000
SIG41: -0.0000
SIG42: 0.0029
SIG43: -0.0000
SIG44: -0.0000
SIG45: 0.0000
SIG46: 0.0000
SIG47: -0.0000
SIG48: -0.0000
SIG49: -0.0000
SIG50: 0.0103
SIG51: 0.0000
SIG52: -0.0000
SIG53: -0.0000
SIG54: -0.0000
SIG55: -0.0000
SIG56: -0.0000
SIG57: -0.0050
SIG58: -0.0000
SIG59: 0.0007
SIG60: -0.0000
SIG61: -0.0128
SIG62: -0.0000
SIG63: 0.0000
SIG64: 0.0008
SIG65: -0.0000
SIG66: 0.0012
SIG67: -0.0000


In [11]:
coeff_df = pd.DataFrame({'Feature': feature_names, 'Coefficient': coefficients})


# Create an interactive scatter plot using Plotly
fig = px.scatter(coeff_df, x='Feature', y='Coefficient',
                 title='Lasso Regression Coefficients', labels={'Coefficient': 'Coefficient Value'})

# Customize layout
fig.update_layout(xaxis={'tickangle': -45, 'title': 'Feature'},
                  yaxis={'title': 'Coefficient'},
                  showlegend=False)


# Show the interactive plot
fig.show()

In [12]:
lasso_model = Lasso(alpha=0.01)
lasso_model.fit(X1, y0)

# Get the coefficients and feature names for Lasso
lasso_coefficients = lasso_model.coef_
feature_names = X0.columns

# Initialize the OLS regression model
ols_model = LinearRegression()
# Fit the OLS model to the training data
ols_model.fit(X1, y0)

# Get the coefficients for OLS
ols_coefficients = ols_model.coef_

# Create DataFrames for Lasso and OLS coefficients
lasso_coeff_df = pd.DataFrame({'Feature': feature_names, 'Lasso Coefficient': lasso_coefficients})
ols_coeff_df = pd.DataFrame({'Feature': feature_names, 'OLS Coefficient': ols_coefficients})

# Combine the DataFrames
combined_coeff_df = pd.merge(lasso_coeff_df, ols_coeff_df, on='Feature')


In [13]:
# Create an interactive scatter plot using Plotly
fig = px.scatter(combined_coeff_df, x='Feature', y=['Lasso Coefficient', 'OLS Coefficient'],
                 title='Lasso vs OLS Regression Coefficients', labels={'value': 'Coefficient'})

# Customize layout
fig.update_layout(xaxis={'tickangle': -45, 'title': 'Feature'},
                  yaxis={'title': 'Coefficient'})

# Show the interactive plot
fig.show()

## Model selection via AICc

How to choose $\lambda$? A quick summary and more details are coming after.
- AIC = deviance + $2df$
- AICc = deviance + $2df \frac{n}{n-df-1}$
- BIC = deviance + $\log(n) \times df$
Estimate the model once and choose the value $\lambda$ that gives the optimal criterion. The textbook (Taddy) recommends **AICc**.

- These are all **in-sample**. They are fast to compute.


In [14]:
alphas = np.logspace(-5, 1, num=100)

# Calculate AICc for each alpha
aiccs = []
for alpha in alphas:
    lasso_model = Lasso(alpha=alpha)
    lasso_model.fit(X1, y0)
    mse = mean_squared_error(y0, lasso_model.predict(X1))
    n = len(y0)
    num_params = np.count_nonzero(lasso_model.coef_) + 1  # Include the intercept
    aicc = n * np.log(mse) + 2 * num_params + 2 * num_params * (num_params+1) / (n - num_params - 1)
    aiccs.append(aicc)

# Find the alpha with the minimum AICc
best_alpha = alphas[np.argmin(aiccs)]
print(f"Best alpha: {best_alpha}")

Best alpha: 0.006135907273413176


In [15]:
fig = px.scatter(x=alphas, y=aiccs, log_x=True,
                 title='AICc vs Alpha (Penalty Term)',
                 labels={'x': 'Alpha (Penalty Term)', 'y': 'AICc'})

# Show the interactive plot
fig.show()

$\color{red}{Question:}$ Why is it flat on the right tail?

## Logistic Lasso

In [18]:
# Initialize the Logistic Regression model with Lasso regularization
lasso_logreg_model = LogisticRegression(penalty='l1', solver='liblinear', C=500)

# Fit the model to the scaled training data
lasso_logreg_model.fit(X1, y0)

np.sum(lasso_logreg_model.coef_ != 0)

200

In [19]:
Cs = np.logspace(-3, 2, num=100)

# Calculate AICc for each alpha
aiccs = []
for C in Cs:
    lasso_logreg_model = LogisticRegression(penalty='l1', solver='liblinear', C=C)
    lasso_logreg_model.fit(X1, y0)
    pred_prob = lasso_logreg_model.predict_proba(X1)[:,1]
    n = len(y0)
    num_params = np.count_nonzero(lasso_logreg_model.coef_) + 1  # Include the intercept
    aicc = -2 * np.sum(y0 * np.log(pred_prob) + (1 - y0) * np.log(1 - pred_prob)) \
        + 2 * num_params + 2 * num_params * (num_params+1) / (n - num_params - 1)
    aiccs.append(aicc)

In [20]:
# Find the minimum AICc
best_C = Cs[np.argmin(aiccs)]
print(f"Best C: {best_C}")

Best C: 0.05857020818056667


In [21]:
fig = px.scatter(x=Cs, y=aiccs, log_x=True,
                 title='AICc vs C (Inverse Penalty Term)',
                 labels={'x': 'C (Inverse Penalty Term)', 'y': 'AICc'})

# Show the interactive plot
fig.show()

## Cross-Validation

In [22]:
lasso_cv_model = LassoCV(alphas=np.logspace(-5, 2, num=100), cv=5, max_iter=10000)

# Fit the LassoCV model to the scaled training data
lasso_cv_model.fit(X1, y0)


,eps,0.001
,n_alphas,'deprecated'
,alphas,array([1.0000...00000000e+02])
,fit_intercept,True
,precompute,'auto'
,max_iter,10000
,tol,0.0001
,copy_X,True
,cv,5
,verbose,False
,n_jobs,None


In [23]:
# The optimal alpha selected by cross-validation
optimal_alpha = lasso_cv_model.alpha_
print("Optimal Alpha:", optimal_alpha)

Optimal Alpha: 0.01788649529057435


In [24]:
lasso_model = Lasso(alpha=optimal_alpha)
lasso_model.fit(X1, y0)
sum(lasso_model.coef_!=0)

8

In [25]:
cv_results = pd.DataFrame({'alpha': lasso_cv_model.alphas_,
                           'mse': np.mean(lasso_cv_model.mse_path_, axis=1),
                           'std_error': np.std(lasso_cv_model.mse_path_, axis=1)})

fig = px.line(cv_results, x='alpha', y='mse')
fig.update_layout(title='Lasso Cross-Validation Results',
                  xaxis_title='Alpha',
                  xaxis_type='log',
                  yaxis_title='Mean Squared Error')
fig.update_traces(error_y=dict(type='data', array=cv_results['std_error']))


fig.show()

## Pipeline and GridSearchCV 

In [ ]:
pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('model', Lasso(max_iter=10000))  
])

alphas = np.logspace(-5, 1, 100)
param_grid = {'model__alpha': alphas}

grid = GridSearchCV(pipe, param_grid, scoring='neg_mean_squared_error', cv=5)
grid.fit(X1, y0)

# best model and alpha
best_model = grid.best_estimator_
best_alpha = grid.best_params_['model__alpha']
print("Best alpha:", best_alpha)

Best alpha: 0.018738174228603847


Use Pipeline and GridSearchCV to re-do the Logistic Regression model selection.

In [ ]:
pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('model', LogisticRegression(penalty='l1', solver='saga', max_iter=10000))  
])

Cs = np.logspace(-3, 2, num=100)
param_grid = {'model__C': Cs}

grid = GridSearchCV(pipe, param_grid, scoring='accuracy', cv=5, verbose=1, n_jobs=-1) # n_jobs makes parallelisation
grid.fit(X1, y0)

# best model and alpha
best_model = grid.best_estimator_
best_C = grid.best_params_['model__C']
print("Best C:", best_C)

Fitting 5 folds for each of 100 candidates, totalling 500 fits
Best C: 0.001


## Elastic net

In [9]:
alpha = .5
l1_ratio = 0.5
elastic_net = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=42)
elastic_net.fit(X1, y0)
y_pred = elastic_net.predict(X1)
mse = mean_squared_error(y0, y_pred)
# Print results
print("Elastic Net Regression Results")
print("-" * 30)
print(f"Alpha: {alpha}")
print(f"L1 Ratio: {l1_ratio}")
print(f"Mean Squared Error: {mse:.2f}")
print("\nCoefficients:")
coef_names = [f"coef_{i+1}" for i in range(len(elastic_net.coef_))]
coef_values = [f"{coef:.4f}" for coef in elastic_net.coef_]
coef_df = pd.DataFrame({"Coefficient": coef_names, "Value": coef_values})
print(coef_df)
print("\nIntercept:", elastic_net.intercept_)
print("\nnon-zero values:", np.sum(elastic_net.coef_!=0))

Elastic Net Regression Results
------------------------------
Alpha: 0.5
L1 Ratio: 0.5
Mean Squared Error: 0.06

Coefficients:
    Coefficient    Value
0        coef_1  -0.0000
1        coef_2  -0.0000
2        coef_3   0.0000
3        coef_4  -0.0000
4        coef_5   0.0000
..          ...      ...
195    coef_196  -0.0000
196    coef_197  -0.0000
197    coef_198   0.0000
198    coef_199  -0.0000
199    coef_200  -0.0000

[200 rows x 2 columns]

Intercept: 0.06770480704129993

non-zero values: 0


In [10]:
# elastic net and cross-validation
alphas = np.logspace(-6, 2, 100)  # Range of alpha values to test
l1_ratios = [0.1, 0.5, 0.7, 0.9, 0.95, 0.99, 1]  # Range of l1_ratio values to test

elastic_net = ElasticNetCV(alphas=alphas, l1_ratio=l1_ratios, cv=5, max_iter=10000, n_jobs=-1)
elastic_net.fit(X1, y0)

best_alpha = elastic_net.alpha_
best_l1_ratio = elastic_net.l1_ratio_

In [11]:
print(best_alpha, best_l1_ratio)

0.019179102616724886 0.99


# Browser Data

More realistic now...

In [12]:
df_web = pd.read_csv("https://www.dropbox.com/s/5fxm63zct40a6b2/browser-domains.csv?dl=1")
df_web.head()

,id,site,visits
0,991,873,1
1,7940,873,2
2,2453,873,12
3,1650,873,1
4,1290,873,6


In [ ]:
df_site = pd.read_csv("https://www.dropbox.com/s/upwat43m9ua1m79/browser-sites.txt?dl=1", sep="\t", header=None)
df_site.columns = ['site']
df_site.index = df_site.index+1 #Blame R indexing
df_site

,site
1,atdmt.com
2,yahoo.com
3,whenu.com
4,weatherbug.com
5,msn.com
...,...
996,effectivebrand.com
997,dallascowboys.com
998,leadgenetwork.com
999,in.us


## Working with multiple datasets

In [14]:
replace_dict = dict(zip(df_site.index, df_site['site']))
replace_dict

{1: 'atdmt.com',
 2: 'yahoo.com',
 3: 'whenu.com',
 4: 'weatherbug.com',
 5: 'msn.com',
 6: 'google.com',
 7: 'aol.com',
 8: 'questionmarket.com',
 9: 'googlesyndication.com-o02',
 10: 'casalemedia.com',
 11: 'mywebsearch.com',
 12: 'myspace.com',
 13: 'pointroll.com',
 14: 'atwola.com',
 15: 'yieldmanager.com',
 16: 'live.com',
 17: 'aim.com',
 18: 'mediaplex.com',
 19: 'precisionclick.com',
 20: 'tribalfusion.com',
 21: 'insightexpressai.com',
 22: 'trafficmp.com',
 23: 'ebay.com',
 24: 'realmedia.com',
 25: 'zedo.com',
 26: 'advertising.com',
 27: 'microsoft.com',
 28: 'hotbar.com',
 29: 'adrevolver.com',
 30: 'ru4.com',
 31: '180solutions.com',
 32: 'nextag.com',
 33: 'accuweather.com',
 34: 'overture.com',
 35: 'hotmail.com',
 36: 'passport.com',
 37: 'my-etrust.com',
 38: 'starware.com',
 39: 'relevantknowledge.com',
 40: 'myway.com',
 41: 'partner2profit.com',
 42: 'ditto.com',
 43: 'kanoodle.com',
 44: 'ebayobjects.com',
 45: 'mcafee.com',
 46: 'comcast.net',
 47: 'fastclick.ne

In [15]:
df_web['site'] = df_web['site'].replace(replace_dict)
df_web

,id,site,visits
0,991,032439.com,1
1,7940,032439.com,2
2,2453,032439.com,12
3,1650,032439.com,1
4,1290,032439.com,6
...,...,...,...
2271679,8767,zwire.com,1
2271680,3786,zwire.com,32
2271681,5907,zwire.com,1
2271682,5345,zwire.com,14


- id is browser/machine, or person/household loosely
- site: website visited
- visits: number of visits

In [16]:
df_web.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2271684 entries, 0 to 2271683
Data columns (total 3 columns):
 #   Column  Dtype 
---  ------  ----- 
 0   id      int64 
 1   site    object
 2   visits  int64 
dtypes: int64(2), object(1)
memory usage: 52.0+ MB


In [37]:
# this step is for convenient data loading. Not necessary
df_web.to_csv("sy_df_web.csv", index=False)

In [38]:
df_web = pd.read_csv("https://www.dropbox.com/s/fvw0y31s1yk19fa/sy_df_web.csv?dl=1")

In [18]:
id_visit_sums = df_web.groupby("id")["visits"].sum()

normalized_visits = []
for id, visits in zip(df_web["id"], df_web["visits"]):
    normalized_visits.append(100 * visits / id_visit_sums[id])
df_web["Normalized_visits"] = normalized_visits
df_web.head()

,id,site,visits,Normalized_visits
0,991,032439.com,1,0.029638
1,7940,032439.com,2,0.017897
2,2453,032439.com,12,0.130890
3,1650,032439.com,1,0.015538
4,1290,032439.com,6,0.088666


## Pivoting

[<img src="https://www.dropbox.com/s/rxp8vxn4lcldx3u/pivot.png?dl=1" width = 600>](https://youtu.be/8w3wmQAMoxQ)

In [19]:
matrix = df_web.pivot(index='id', columns='site', values='Normalized_visits').fillna(0)

In [20]:
matrix.loc[301, 'nbc.com']

0.048216007714561235

In [21]:
matrix.head()

site,032439.com,123greetings.com,157.22.32.111,180solutions.com,192.168.1.1,204.181.57.155,204.95.60.12,207.97.212.250,216.133.243.28,216.139.222.230,...,yournewsletters.net,youtube.com,ysbweb.com,zango.com,zap2it.com,zappos.com,zedo.com,zonelabs.com,ztod.com,zwire.com
id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.10005,0.0,0.0,0.0,0.000000,0.025013,0.0,0.0,0.0,...,0.0,0.075038,0.025013,0.0,0.025013,0.0,0.625313,0.0,0.0,0.000000
2,0.0,0.00000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.064475,0.000000,0.0,0.000000,0.0,0.644745,0.0,0.0,0.000000
3,0.0,0.00000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.019489,0.000000,0.0,0.019489,0.0,0.389788,0.0,0.0,0.000000
4,0.0,0.00000,0.0,0.0,0.0,0.022655,0.000000,0.0,0.0,0.0,...,0.0,0.000000,0.022655,0.0,0.113276,0.0,1.631174,0.0,0.0,0.022655
5,0.0,0.00000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.930774,0.0,0.0,0.000000


## GIGO
As a sanity check, see what sites household #1 visited

- This is actually the **most important** step
- remember: **garbage in, garbage out**, or [GIGO](https://en.wikipedia.org/wiki/Garbage_in,_garbage_out)

In [ ]:
matrix.loc[1, matrix.loc[1] != 0].sort_values(ascending=False) # can you read and understand what is going on here?

site
cox.net               13.381691
yahoo.com             11.855928
google.com             6.528264
dell.com               4.777389
atdmt.com              4.052026
                        ...    
myinsiderdeals.com     0.025013
mymailstamp.com        0.025013
checkm8.com            0.025013
mywebface.com          0.025013
kontera.com            0.025013
Name: 1, Length: 302, dtype: float64

In [23]:
df_spend = pd.read_csv("https://www.dropbox.com/s/pmh3l3qsqvv05c3/browser-totalspend.csv?dl=1")
df_spend.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   id      10000 non-null  int64
 1   spend   10000 non-null  int64
dtypes: int64(2)
memory usage: 156.4 KB


In [24]:
df_spend.head()

,id,spend
0,1,424
1,2,2335
2,3,279
3,4,829
4,5,221


## LASSO it

In [25]:
lasso_model = Lasso(alpha=10)  # You can adjust the alpha parameter for stronger or weaker regularization

lasso_model.fit(matrix, df_spend['spend'])


,alpha,10
,fit_intercept,True
,precompute,False
,copy_X,True
,max_iter,1000
,tol,0.0001
,warm_start,False
,positive,False
,random_state,None
,selection,'cyclic'


In [26]:
print(np.sum(lasso_model.coef_ != 0))
yhat = lasso_model.predict(matrix)
print(mean_squared_error(df_spend['spend'], yhat))

323
58081076.26407346


In [27]:
n = 10000
num_params = 323
aicc = n * np.log(mean_squared_error(df_spend['spend'], yhat)) \
+ 2 * num_params + 2 * num_params * (num_params+1) / (n - num_params - 1)
print(aicc)

179441.1358428501


In [29]:
alphas = np.logspace(-2, 3, num=50)
scaler = StandardScaler()

X = scaler.fit_transform(matrix) #normlisation
y = df_spend['spend']


# Calculate AICc for each alpha
aiccs = []
for alpha in alphas:
    lasso_model = Lasso(alpha=alpha, max_iter=10000)
    lasso_model.fit(X, y)
    mse = mean_squared_error(y, lasso_model.predict(X))
    n = len(y)
    num_params = np.count_nonzero(lasso_model.coef_) + 1  # Include the intercept
    aicc = n * np.log(mse) + 2 * num_params + 2 * num_params * (num_params+1) / (n - num_params - 1)
    aiccs.append(aicc)

# Find the alpha with the minimum AICc
best_alpha = alphas[np.argmin(aiccs)]
print(f"Best alpha: {best_alpha}")

/Users/yongs1/miniconda3/envs/ada-env/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.392e+08, tolerance: 6.461e+07
  model = cd_fast.enet_coordinate_descent(


Best alpha: 120.67926406393289


In [32]:
fig = px.scatter(x=alphas, y=aiccs, log_x=True,
                 title='AICc vs Alpha (Penalty Term)',
                 labels={'x': 'Alpha (Penalty Term)', 'y': 'AICc'})

# Show the interactive plot
fig.show()

In [34]:
lasso_model = Lasso(alpha=30)  # You can adjust the alpha parameter for stronger or weaker regularization

lasso_model.fit(X, df_spend['spend'])


,alpha,30
,fit_intercept,True
,precompute,False
,copy_X,True
,max_iter,1000
,tol,0.0001
,warm_start,False
,positive,False
,random_state,None
,selection,'cyclic'


In [36]:
print(np.sum(lasso_model.coef_ != 0))
yhat = lasso_model.predict(X)
print(mean_squared_error(df_spend['spend'], yhat))

495
56794942.82089353


In [37]:
n = 10000
num_params = 122
aicc = n * np.log(mean_squared_error(df_spend['spend'], yhat)) \
+ 2 * num_params + 2 * num_params * (num_params+1) / (n - num_params - 1)
print(aicc)

178796.61702330713


## cross validation

In [38]:
lasso_cv_model = LassoCV(alphas=np.logspace(0, 2, num=30), cv=5)

# Fit the LassoCV model to the data
lasso_cv_model.fit(X, df_spend['spend'])

,eps,0.001
,n_alphas,'deprecated'
,alphas,array([ 1. ...100. ])
,fit_intercept,True
,precompute,'auto'
,max_iter,1000
,tol,0.0001
,copy_X,True
,cv,5
,verbose,False
,n_jobs,None


In [39]:
lasso_cv_model.alpha_

100.0

In [41]:
lasso_model = Lasso(alpha=73)  # You can adjust the alpha parameter for stronger or weaker regularization

lasso_model.fit(X, df_spend['spend'])


,alpha,73
,fit_intercept,True
,precompute,False
,copy_X,True
,max_iter,1000
,tol,0.0001
,warm_start,False
,positive,False
,random_state,None
,selection,'cyclic'


In [42]:
print(np.sum(lasso_model.coef_ != 0))
yhat = lasso_model.predict(X)
print(mean_squared_error(df_spend['spend'], yhat))

193
58410166.852406815


In [43]:
np.nonzero(lasso_model.coef_ != 0)

(array([  4,  11,  15,  21,  25,  29,  54,  68,  75,  82,  87,  88,  90,
         95, 101, 103, 104, 116, 124, 125, 132, 133, 137, 140, 145, 148,
        153, 156, 160, 162, 170, 173, 175, 183, 204, 206, 221, 228, 230,
        233, 234, 238, 240, 241, 242, 254, 256, 257, 263, 268, 289, 296,
        297, 303, 308, 310, 325, 327, 333, 351, 354, 356, 372, 378, 389,
        391, 395, 397, 399, 400, 403, 409, 430, 434, 440, 445, 448, 461,
        466, 474, 481, 482, 483, 484, 487, 503, 508, 511, 515, 520, 526,
        527, 528, 535, 541, 542, 543, 555, 558, 566, 573, 576, 577, 582,
        588, 592, 593, 608, 609, 615, 625, 629, 641, 649, 653, 663, 670,
        674, 678, 679, 680, 682, 684, 687, 688, 698, 709, 714, 718, 722,
        724, 731, 733, 739, 741, 743, 744, 753, 754, 755, 759, 761, 765,
        770, 771, 773, 779, 780, 783, 785, 786, 789, 790, 791, 808, 810,
        815, 818, 819, 828, 834, 837, 852, 855, 866, 867, 875, 879, 880,
        882, 893, 898, 901, 903, 907, 911, 916, 922

In [44]:
matrix.columns[np.nonzero(lasso_model.coef_ != 0)]

Index(['192.168.1.1', '2o7.net', '64.136.28.49', 'aa.com', 'about.com',
       'active.com', 'advertising.com', 'alt.com', 'amazon.com', 'andale.com',
       ...
       'webmd.com', 'webmd.com-o01', 'wellsfargo.com', 'wikipedia.org',
       'winantivirus.com', 'windowsmedia.com', 'winfixer.com', 'yahoo.com-o08',
       'yellowpages.com', 'zappos.com'],
      dtype='object', name='site', length=193)

$\color{red}{Question:}$ sort the coefficients and find the most influential site on expenditure.

# Summary

- Model: linear model or generalised linear model such as the Logistic regression model.

- Aim: control for overparametrization.

- Method: LASSO, Ridge or ElasticNet.

- Choice of Penalty paramter: AICc (AIC, BIC) or cross-validation.

# Exercise: Orange Juice

- LASSO is not everything.

In [45]:
df_oj = pd.read_csv("https://www.dropbox.com/s/8oll01xv92mkij8/oj.csv?dl=1")
df_oj

,sales,price,brand,feat
0,8256.0,3.87,tropicana,0
1,6144.0,3.87,tropicana,0
2,3840.0,3.87,tropicana,0
3,8000.0,3.87,tropicana,0
4,8896.0,3.87,tropicana,0
...,...,...,...,...
28942,2944.0,2.00,dominicks,0
28943,4928.0,1.94,dominicks,0
28944,13440.0,1.59,dominicks,0
28945,55680.0,1.49,dominicks,0


## brand dummy and OLS

In [46]:
df_oj_dummy = pd.get_dummies(df_oj, columns=['brand'])
df_oj_dummy = df_oj_dummy.rename(columns={'brand_minute.maid':'brand_minute_maid'})
df_oj_dummy

,sales,price,feat,brand_dominicks,brand_minute_maid,brand_tropicana
0,8256.0,3.87,0,False,False,True
1,6144.0,3.87,0,False,False,True
2,3840.0,3.87,0,False,False,True
3,8000.0,3.87,0,False,False,True
4,8896.0,3.87,0,False,False,True
...,...,...,...,...,...,...
28942,2944.0,2.00,0,True,False,False
28943,4928.0,1.94,0,True,False,False
28944,13440.0,1.59,0,True,False,False
28945,55680.0,1.49,0,True,False,False


In [47]:
# Create the formula for the OLS model
formula = 'I(np.log(sales)) ~  price + feat + brand_minute_maid	+ brand_tropicana'

# Fit the OLS model using the formula and data
model = sm.OLS.from_formula(formula, data=df_oj_dummy)
results = model.fit()

print(results.summary())

                            OLS Regression Results                            
Dep. Variable:       I(np.log(sales))   R-squared:                       0.482
Model:                            OLS   Adj. R-squared:                  0.482
Method:                 Least Squares   F-statistic:                     6743.
Date:                Thu, 07 Aug 2025   Prob (F-statistic):               0.00
Time:                        13:57:32   Log-Likelihood:                -32098.
No. Observations:               28947   AIC:                         6.421e+04
Df Residuals:                   28942   BIC:                         6.425e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                                coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------
Intercept             

## Alternative dummy method

In [48]:
# Create the formula for the OLS model
formula = 'I(np.log(sales)) ~  price +feat + brand'

# Fit the OLS model using the formula and data
model = sm.OLS.from_formula(formula, data=df_oj)
results = model.fit()

print(results.summary())

                            OLS Regression Results                            
Dep. Variable:       I(np.log(sales))   R-squared:                       0.482
Model:                            OLS   Adj. R-squared:                  0.482
Method:                 Least Squares   F-statistic:                     6743.
Date:                Thu, 07 Aug 2025   Prob (F-statistic):               0.00
Time:                        13:57:39   Log-Likelihood:                -32098.
No. Observations:               28947   AIC:                         6.421e+04
Df Residuals:                   28942   BIC:                         6.425e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
Intercept               10.7606 

## Interactions

In [49]:
# Create the formula for the OLS model
formula = 'I(np.log(sales)) ~  price*brand + feat'

# Fit the OLS model using the formula and data
model = sm.OLS.from_formula(formula, data=df_oj)
results = model.fit()

print(results.summary())

                            OLS Regression Results                            
Dep. Variable:       I(np.log(sales))   R-squared:                       0.509
Model:                            OLS   Adj. R-squared:                  0.509
Method:                 Least Squares   F-statistic:                     5008.
Date:                Thu, 07 Aug 2025   Prob (F-statistic):               0.00
Time:                        13:57:49   Log-Likelihood:                -31322.
No. Observations:               28947   AIC:                         6.266e+04
Df Residuals:                   28940   BIC:                         6.272e+04
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept           

## More interactions

In [50]:
# Create the formula for the OLS model
formula = 'I(np.log(sales)) ~  price*feat*brand'

# Fit the OLS model using the formula and data
model = sm.OLS.from_formula(formula, data=df_oj)
results = model.fit()

print(results.summary())

                            OLS Regression Results                            
Dep. Variable:       I(np.log(sales))   R-squared:                       0.530
Model:                            OLS   Adj. R-squared:                  0.530
Method:                 Least Squares   F-statistic:                     2967.
Date:                Thu, 07 Aug 2025   Prob (F-statistic):               0.00
Time:                        13:57:55   Log-Likelihood:                -30699.
No. Observations:               28947   AIC:                         6.142e+04
Df Residuals:                   28935   BIC:                         6.152e+04
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
                                      coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------
Intercept 

In [51]:
model.exog_names

['Intercept',
 'brand[T.minute.maid]',
 'brand[T.tropicana]',
 'price',
 'price:brand[T.minute.maid]',
 'price:brand[T.tropicana]',
 'feat',
 'feat:brand[T.minute.maid]',
 'feat:brand[T.tropicana]',
 'price:feat',
 'price:feat:brand[T.minute.maid]',
 'price:feat:brand[T.tropicana]']

## LASSO with CV

In [52]:
X = model.exog
X = np.delete(X, 0, axis=1)
y = model.endog

In [53]:
lasso_cv_model = LassoCV(alphas=np.logspace(-5, 2, num=100), cv=5, max_iter=10000)
# Fit the LassoCV model to the data
lasso_cv_model.fit(X, y)

,eps,0.001
,n_alphas,'deprecated'
,alphas,array([1.0000...00000000e+02])
,fit_intercept,True
,precompute,'auto'
,max_iter,10000
,tol,0.0001
,copy_X,True
,cv,5
,verbose,False
,n_jobs,None


In [54]:
lasso_cv_model.alpha_

1e-05

In [55]:
cv_results = pd.DataFrame({'alpha': lasso_cv_model.alphas_,
                           'mse': np.mean(lasso_cv_model.mse_path_, axis=1),
                           'std_error': np.std(lasso_cv_model.mse_path_, axis=1)})

fig = px.line(cv_results, x='alpha', y='mse')
fig.update_layout(title='Lasso Cross-Validation Results',
                  xaxis_title='Alpha',
                  xaxis_type='log',
                  yaxis_title='Mean Squared Error')
fig.update_traces(error_y=dict(type='data', array=cv_results['std_error']))


fig.show()

## Best LASSO

In [57]:
lasso_model = Lasso(alpha=1e-5, max_iter=10000)  # You can adjust the alpha parameter for stronger or weaker regularization

lasso_model.fit(X, y)

,alpha,1e-05
,fit_intercept,True
,precompute,False
,copy_X,True
,max_iter,10000
,tol,0.0001
,warm_start,False
,positive,False
,random_state,None
,selection,'cyclic'


In [58]:
np.nonzero(lasso_model.coef_)

(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10]),)

In [59]:
lasso_model.coef_

array([-0.90736507, -0.57648114, -1.55149764,  0.72634244,  0.83615373,
        1.67582164,  1.62387594,  0.71884213, -0.48065016, -0.56065251,
       -0.24142485])

In [60]:
lasso_model.intercept_

11.630286862480695